# Backtracking algorithm

The branch and bound algorithm is very useful whenever you have to solve a maximisation/minimisation problem with constraints, such as the knapsack problem. 

The <b>backtracking algorithm</b> is basically the same as the branch and bound but without the function to maximise/minimise. Therefore the algorithm chooses as branch to explore always the deepest one, i.e. the one closer to the possible ending.

A problem for which it can be applied easily is the timetable problem, for which is it is for sure more efficient than the brute-force approach and, probably, also the optimized fuzzy approach.

Let's review the problem:
- a weekly scheduling 
- two hours slot, only 8-10, 10-12, 12-14, 14-16, 16-18
- from 8:00 to 18:00 
- from Monday to Friday 
- 4 courses (C, F, L, S) to accomodate in the timetable.

and the constraints:
- each course exactly 6 hours in the week (it's not true), i.e. three slots
- no lesson on Monday morning, i.e. slots first and second must be 0;
- maximum 1 slot every day for each course
- C never lesson on two consecutive days
- F and Lcan never meet, i.e. their lessons cannot be consecutive
- at least one slot free for lunch among 10-12, 12-14 and 14-16.

## Backtracking example

Let'a apply the backtracking algorithm to the timetable problem. 

We start with an empty timetable and that is our first branch and only branch at depth 0. 

We take it and branch into five branches at depth 1, each one with a different assignment for slot 1 of day 1: nobody, Coletti, Fedele, Lechner, Simonelli. We check them against our constraints and we see that already C, F, L, S fail to satisfy our "no lesson on Monday morning" constraint and thus they are deleted. The only remaining branch in the tree is nobody at slot 1 and it is a branch at depth 1.

<img src="http://www.paolocoletti.it/algorithmicthinking/images/backtracking1.jpg">

We branch it into five branches at depth 2, each one with a different assignment for slot 2 of day 1: nobody, C, F, L, S. We check them against our constraints and we see that already Coletti, Fedele, Lechner, Simonelli fail to satisfy our "no lesson on Monday morning" constraint and thus they are deleted. The only remaining branch in the tree is nobody at slot 1 and nobody at slot 2 and it is a branch at depth 2.

<img src="http://www.paolocoletti.it/algorithmicthinking/images/backtracking2.jpg">

We branch it into five branches at depth 3, each one with a different assignment for slot 3 of day 1: nobody, C, F, L, S. We check them against our constraints and we see that all of them satisfy the constraints. Thus we currently have 5 branches in the tree all at depth 3.

<img src="http://www.paolocoletti.it/algorithmicthinking/images/backtracking3.jpg">

At this point we take randomly one of them (we could also take one of them using a rule if you have any heuristic rule that helps you, perhaps you can take the most problematic professor first which in this example is Coletti). For example let's take the branch with nobody, nobody, Lechner. We branch it into five branches at depth 4, each one with a different assignment for slot 4 of day 1: nobody, C, F, L, S. We check them against our constraints and we see that all of them satisfy the constraints, except putting L again and putting Fedele. So we delete the L's and F's branches and currently we have 4 branches in the tree at depth 3 and 3 branches at depth 4.

<img src="http://www.paolocoletti.it/algorithmicthinking/images/backtracking4.jpg">

We take always the deepest branch (this is a <b>depth-first algorithm</b>), i.e. one of the 3 branches at depth 4. For example, let's take the one with nobody, nobody, L, S. We branch it into five branches and check their validiy. Only nobody, C and F are valid (if you wrote the condition on C as "C never on two consecutive days". On the other hand, if you wrote is as "always on Monday, Wednesday and Friday" only C's branch is valid). We now have 4 branches at depth 3, 2 branches at depth 4 and 3 branches at depth 5. 

<img src="http://www.paolocoletti.it/algorithmicthinking/images/backtracking5.jpg">

We take one of the three branches at depth 5 and...

Sooner or later you will probably end up with all the branches which are not valid and you will have to take into consideration branches at one upper depth. That's why it is called "<u>back</u>tracking".

<img src="http://www.paolocoletti.it/algorithmicthinking/images/backtrackingAll.jpg">


## Backtracking description

Let's describe the algorithm theoretically.

First of all the algorithm needs a procedure which checks the validity of a partial solution. The algorithm starts with an empty solution, where nothing has been decided and which obviously must be valid. This is the branch at depth 0.

Then the algorithm starts its main loop. It takes the deepest open branch. In case there are more branches at the maximum depth it takes one randomly or following any heuristic rule you like. The algorithm considers one of the possible still unsolved microproblems of this branch (in our example it is "who teaches at day X slot Y") and creates all the possible partial solutions. Each one is checked for its validity and discarded if not valid. The valid ones are inserted as open branches at depth 1 more.

When the algorithm analyses a branch for which no more microproblem is open (in our example all the timeslots have been assigned) this is a leaf. It can be easily recognized as it has reached a depth equal to the number of microproblems (in our example 25). It checks the validity of the leaf using the check procedure for a complete solution, as there may be constraints which can only be checked when the timetable is complete (in our example the fact that each professor teaches 3 hours per day). If it is not valid, it is cancelled and the algorithm goes on. If it is valid, it is a solution.

### Complexity

Exactly as for the branch and bound, the backtracking algorithm does not reduce the complexity in the worst case. If your problem is such nasty that all the constraints can be checked only on the final solution, you will have to go everytime up to the leaf which corresponds to a brute-force approach. 

However, many problems allow you to check also the validity of partial solutions. You can see in the example above that cutting 4 branches at depth 1 reduces the complexity to 20% and cutting other 4 branches at depth 2 further reduced the complexity to 20%. Only with these two steps the complexity is now 4% of the original one. 

Therefore it is important to put as many constraints as possible in the procedure which checks the validity of partial solutions, to make it work and cut useless branches as soon as possible.

## The algorithm

### Data structure

We shall indicate each partial timetable TT as <font color=GREEN><b>[ [0,0,3,4,1] , [2,4] , [ ] , [ ] , [ ] ]</b></font> . Obviously, unless you are looking for troubles, the assigned slots will be the first ones and the unassigned will be the last ones. 

Therefore, together with TT, it is useful to indicate also the depth (in this case 7), the last day in which we have assignments (in this case 1) and the last slot for which we have assignments (in this case 1). These data can be deduces from the partial timetable, buy having them handy will make our life easier for the ckeckPartial procedure below.

Thus our structure BT will be a list which contains elements such as <font color=BLUE>[depth, last filled day, last filled slot of that day, TT]</font>, in our example: <font color=GREEN><b>[ 7 , 1 , 1 , [ [0,0,3,4,1] , [2,4], [ ] , [ ] , [ ] ] ]</b></font> . 

### Check procedure

We need to rite two check procedures here. One is the one that you also used for the brute force approach and is used only to check the validity of the leaves. The other procedure must be able to check the validity of partial solution, so:
- some constraints can be checked only on the final timetable and these ones are skipped
- some constraints can be in part checked on the partial timetable and it is worthwhile to modify them to check whatever you can check, because any restriction you check early will cut useless branches
- some constraints can be fully checked also on the partial timetable, probably paying attention that some slots now can be unassigned

Take your own original check procedure. Keep it and build another checkPartial procedure which takes as input a partial timetable, its last filled day and its last filled slot and checks its validity. Pay attention that it must take care that some slots are unassigned. 

In [ ]:
def check(TT):
    # respect lunch break
    for anyday in TT :
        if (anyday[1] != 0) and (anyday[2] != 0) and (anyday[3] != 0) :
            print("Attention: no lunch break on day ", anyday)
            return False
    
    # no lesson on Monday morning
    if (TT[0][0] != 0) or (TT[0][1] != 0):
        print("Attention: lesson on Monday morning")
        return False

    # each course exactly 3 slots in the week
    # we have to go through the entire week and find out how many slots
    # does each lecturer have
    slots={ 0:0, 1:0, 2:0, 3:0, 4:0 }
    day=0
    while day<=4:
        hour=0
        while hour<=4:
            currentProf=TT[day][hour]
            slots[currentProf]=slots[currentProf]+1
            hour=hour+1
        day=day+1
    # now inside slots I have how many slots does each prof have
    for x in slots:
        if x>0 and slots[x]!=3:
            print("Attention! Lecturer ",x," has ",slots[x]," slots")
            return False
    
    # maximum 1 slot every day for each course
    for d in TT :
        perday = []
        for e in d :
            if e in perday and e != 0 :
                print("max 1 slot every day for each course")
                return False
            perday.append(e)

    # C never lesson on two consecutive days
    for k in range(0,4) :
        if 1 in TT[k] and 1 in TT[k+1] :
            print("C lessons on two consecutive days")
            return False
    
    # F and L can never meet, i.e. their lessons cannot be consecutive
    for m in TT :
        for c in range(0,4) :
            if (m[c] == 2 and m[c+1] == 3) or (m[c] == 3 and m[c+1] == 2) :
                print("F and L met")
                return False
    return True

def checkPartial(day,slot,TT) :
# write here a modified check procedure, which checks things up to day-1 and for day it checks only up to slot
# obvisouly some checks cannot be done while others need to be adjusted
    
    # partial : respect lunch break
    if len(TT[0]) > 3 :
        for x in range(0, day) :
            if (TT[x][1] != 0) and (TT[x][2] != 0) and (TT[x][3] != 0) :
                print("Attention: no lunch break on")
                return False
    if len(TT[day]) > 3 :
        if (TT[day][1] != 0) and (TT[day][2] != 0) and (TT[day][3] != 0) :
            print("Attention: no lunch break on")
            return False
        
    # partial : no lesson on Monday morning
    if len(TT[0]) == 1 :
        if (TT[0][0] != 0) :
            print("Attention: lesson on Monday morning")
            return False
    if len(TT[0]) > 1 :
        if (TT[0][0] != 0) or (TT[0][1] != 0):
            print("Attention: lesson on Monday morning")
            return False
    
    # partial : maximum 1 slot every day for each course
    for e in range(0, day) :
        perday = []
        for x in range(0,5) :
            if TT[e][x] in perday and TT[e][x]!= 0 :
                print("max 1 slot every day for each course")
                return False
            perday.append(TT[e][day])
    
    if slot > 0 :
        perday = []
        for e in range(0, slot + 1) :
            if TT[day][e] in perday and TT[day][e] != 0 :
                print("max 1 slot every day for each course")
                return False
            perday.append(TT[day][e])
    
    # partial : C never lesson on two consecutive days
    if day != 0 :
        for aday in range(0, day) :
            if (1 in TT[aday] ) and (1 in TT[aday + 1]) :
                print("C shouldn't lessons on two consecutive days, but he does")
                return False
    
    # partial : F and L can never meet, i.e. their lessons cannot be consecutive
    for x in range(0, day) :
        for y in range(0, 4) :
            if (TT[x][y] == 2 and TT[x][y + 1] == 3) or (TT[x][y] == 3 and TT[x][y + 1] == 2) :
                print("F and L shoudn't meet, but they do")
                return False
    if slot > 0 :
        for y in range(0, slot) :
            if (TT[day][y] == 2 and TT[day][y + 1] == 3) or (TT[day][y] == 3 and TT[day][y + 1] == 2) :
                print("F and L shoudn't meet, but they do")
                return False
                
    return True


Do not forget to test it!

In [ ]:
print(checkPartial(0,0,[ [0],[],[],[],[] ]))
print(checkPartial(0,3,[ [0,0,4,1],[],[],[],[] ]))
print(checkPartial(1,1,[ [0,0,3,2,1] , [2,4],[],[],[] ]))
print(checkPartial(1,1,[ [0,0,3,1,2] , [2,4],[],[],[] ]))
print(checkPartial(1,4,[ [0,0,3,1,2] , [2,4,0,0,1],[],[],[] ]))

Let's build here the algorithm!

In [ ]:
import copy
import random 

def insertBT(BT, depth, day, slot, TT):
    # this function inserts [depth,day,slot,TT] into the data structure BT in its right place, 
    # keeping BT sorted from largest depth to smallest. 
    # In case you must insert an element with the same depth as an existing one, it doesn't matter whether inserting it before or after

    # how do you make it? Well, copy my insertBB function and modify it to adapt it to the current situation.
    # that's how 95% of modern programs are written....
    i=0
    print("Insert [",depth,day,slot,TT, "] into",BT, end=" ") 
    while i<len(BT):
        if BT[i][0]<depth:
            BT.insert(i,[depth,day,slot,TT])
            print(" and getting", BT,"\n")
            return
        i=i+1
    BT.append([depth,day,slot,TT])
    print("and getting ", BT,"\n")

# we set up the first branch, i.e. the ROOT, with depth 0, current filled day -1, current filled slot 4 
# (setting -1 and 4 will help you branching correctly into branches with current day 0 and current slot 0)

BT=[[0, -1, 4, [[], [], [], [], []]]]
prof=[0, 1, 2, 3, 4]
# now we loop and write the core of the algorithm
while True:
# we take the first element out from the list and store it in a variable. Do not forget that the element is made up of 
# [depth,day,slot,TT]
# please write the code here
    first = BT.pop()
# we check whether the element we just took out is a leaf, i.e. depth is 25
# In this case, we check its validity and if it is a valid complete solution we print the result. 
# If it is not valid, we have already popped it out and we just go back with the while loop

# please write the code here
    if first[0] == 25:
        the_table = first[3]
        valid = check(the_table)
        if valid :
            print(the_table)
            
    # else when the first element of BT is not a leaf, we have to branch 
    # building all the branches and inserting them in the data structure BT (if they are valid)
    # pay attention when building TT that if slot is equal to 4, you will have to insert a new day

    # NOTE: if for any reason you need to copy a list, beware that a=b is not a duplication of the list but only of the pointer
    # to copy a list duplicating it, use  a=copy.deepcopy(b)  In this way a and b will be two different objects using two different spaces in the memory

    # please write the code here
    depth = copy.deepcopy(first[0]) + 1
    day = copy.deepcopy(first[1])
    slot = copy.deepcopy(first[2]) + 1
    if slot > 4 :
        day = copy.deepcopy(first[1]) + 1
        slot = 0
    for x in range(4, -1, -1) :
        TT = copy.deepcopy(first[3])
        TT[day].append(x)
        valid_partial = checkPartial(day, slot, TT)
        if valid_partial :
            insertBT(BT, depth, day, slot, TT)
            

### Optimisations

The biggest problem when running the above code is that when you have to insert something like [2,0,2,[0,0,1] ] in a structure such as [ [2,0,2,[0,0,0] ]] . They have the same depth. Do you insert it before or after? 

If you insert it after, you will have as first place most of the times a structure with several zeroes and you will see that your algorithm quickly arrives with a timetable full of zeroes and only at that point it realises that there are too few slots for professors 1,2,3,4 and starts to backtrack trying to put them somewhere.

If you insert it before, you will have as first place most of the times S. This is much better, as S cannot go in more than 3 slots (i.e. the algorithm realises very soon that there are non valid branches). 

#### Fuzzy

We randomly choose the place to put a branch, but only among the ones which have the same depth. You have to deduce the value of iStart and iEnd (the indexes where the branches with the same depth start and end) and pick up a random integer number between iStart and iEnd where you will insert the new branch.

Modify the insertBT accordingly and test the algorith with it.

In [ ]:
ef insertBTrandom(BT, depth, day, slot, TT):
    # this function inserts [depth,day,slot,TT] into the data structure BT in its right place, 
    # keeping BT sorted from largest depth to smallest. 
    # In case you must insert an element with the same depth as an existing one, it inserts it randomly
    i=0
    print("Insert [",depth,day,slot,TT, "] into",BT, end=" ") 
    while i<len(BT):
        if BT[i][0]<depth:
            BT.insert(i,[depth,day,slot,TT])
            print(" and getting", BT,"\n")
            return
        i=i+1
    for c in range(len(BT)):
        if BT[c][0] == depth:
            BT.insert(random.randint(0, len(BT)),[depth,day,slot,TT])
            break
    BT.append([depth,day,slot,TT])
    print("and getting ", BT,"\n")
    

#### Most problematic first, then randomly

If the current branch contains C as the last slot, then insert it before all the branches with its same depth. Otherwise, insert it randomly among the branches with the same depth. 

In this way the algorithm will first try to fix C who has the strongest constaint which will eliminate a lot of non valid branches. The rest randomly.

Modify the inserBT accordingly and test the algorithm with it. With this technique I was able to find a solution instantly.

In [ ]:
def insertBTColetti(BT, depth, day, slot, TT):
    # this function inserts [depth,day,slot,TT] into the data structure BT in its right place, 
    # keeping BT sorted from largest depth to smallest. 
    # In case you must insert an element with the same depth as an existing one, 
    # if it is a slot with C at the end it is inserted first, otherwise randomly 

    # if you were not able to do the previous procedure, then if it is not C
    # insert it after all the ones with the same depth
    i=0
    print("Insert [",depth,day,slot,TT, "] into",BT, end=" ") 
    while i<len(BT):
        if BT[i][0]<depth:
            BT.insert(i,[depth,day,slot,TT])
            print(" and getting", BT,"\n")
            return
        i=i+1
    for c in range(len(BT)):
        if BT[c][0] == depth:
            if 1 in BT[len(BT)-1][3]:
                BT.insert(len(BT)-2,[depth,day,slot,TT])     
            else:
                BT.insert(random.randint(0, len(BT)),[depth,day,slot,TT])
            break
    BT.append([depth,day,slot,TT])
    print("and getting ", BT,"\n")